In [52]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet121
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

In [53]:
conv_base = DenseNet121(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [54]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [55]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 9, 9, 1024)

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [56]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [57]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [58]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1024))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)
(59, 9, 9, 1024)


In [59]:
samples = samples.reshape(nos,9*9*1024)

In [60]:
samples.shape, labels.shape

((472, 82944), (472,))

In [61]:
# To save memory
del X
del y

In [62]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 1s 2ms/step - loss: 9.7697 - accuracy: 0.6538 - val_loss: 4.2008 - val_accuracy: 0.7288
Epoch 2/100
413/413 [==============================] - 1s 2ms/step - loss: 7.2352 - accuracy: 0.7506 - val_loss: 4.0315 - val_accuracy: 0.7119
Epoch 3/100
413/413 [==============================] - 1s 2ms/step - loss: 1.6911 - accuracy: 0.8354 - val_loss: 1.1781 - val_accuracy: 0.8136
Epoch 4/100
413/413 [==============================] - 1s 2ms/step - loss: 0.5200 - accuracy: 0.8862 - val_loss: 1.1064 - val_accuracy: 0.7627
Epoch 5/100
413/413 [==============================] - 1s 2ms/step - loss: 0.4021 - accuracy: 0.9153 - val_loss: 1.2307 - val_accuracy: 0.7458
Epoch 6/100
413/413 [==============================] - 1s 2ms/step - loss: 0.2956 - accuracy: 0.9346 - val_loss: 0.7325 - val_accuracy: 0.7288
Epoch 7/100
413/413 [==============================] - 1s 2ms/step - loss: 0.1866 - accura

In [63]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.8441091954022989 0.7468390804597701


Training on whole Training Dataset

In [67]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1024))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [68]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [69]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [70]:
datagentest.fit(xtest)

In [71]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,1024))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 1024)


In [72]:
test_samples = test_samples.reshape(testnos,9*9*1024)

In [73]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 3ms/step - loss: 11.1310 - accuracy: 0.6314 - val_loss: 9.0883 - val_accuracy: 0.5761
Epoch 2/100
472/472 [==============================] - 1s 2ms/step - loss: 3.4271 - accuracy: 0.8072 - val_loss: 5.3799 - val_accuracy: 0.5127
Epoch 3/100
472/472 [==============================] - 1s 2ms/step - loss: 0.8648 - accuracy: 0.8559 - val_loss: 1.2761 - val_accuracy: 0.7989
Epoch 4/100
472/472 [==============================] - 1s 2ms/step - loss: 0.3308 - accuracy: 0.9004 - val_loss: 0.9744 - val_accuracy: 0.8116
Epoch 5/100
472/472 [==============================] - 1s 3ms/step - loss: 0.1535 - accuracy: 0.9492 - val_loss: 0.7227 - val_accuracy: 0.8188
Epoch 6/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1998 - accuracy: 0.9407 - val_loss: 0.8313 - val_accuracy: 0.8116
Epoch 7/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1547 - accuracy: 0

In [74]:
test_preds = model.predict(test_samples, batch_size = 20)

In [75]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [76]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.6440677966101694, 0.8397565922920892)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

TypeError: ignored